# Project Walkthrough (Beginner)

This project follows a structured machine learning workflow to predict credit risk. Here is a 10-step overview of the process:

1.  **Environment Setup**: We load necessary libraries for data manipulation, visualization, and machine learning.
2.  **Data Loading**: Historical loan data is imported from CSV files.
3.  **Exploratory Data Analysis (EDA)**: We examine key variables like interest rates and debt-to-income ratios to understand their relationship with loan defaults.
4.  **Feature Engineering**: New variables are created from existing ones to better capture borrower risk profiles (e.g., loan-to-income ratio).
5.  **Data Splitting**: We split the data into training and validation sets to ensure we can test our models on unseen data.
6.  **Preprocessing Pipeline**: A systematic process is established to handle missing values and convert categorical data into a format machines can understand.
7.  **Model Training**: Multiple models (Logistic Regression, Decision Tree, Random Forest, and Gradient Boosting) are trained and tuned.
8.  **Model Comparison**: We compare the performance of all models using standardized metrics like AUC and RMSE.
9.  **Threshold Optimization**: We determine the best decision-making threshold to balance the cost of defaults against the benefit of approved loans.
10. **Final Export**: All key results, models, and diagnostic charts are exported for reproducibility and reporting.

# Glossary

- **Target / Label**: The outcome we are trying to predict (in this case, whether a loan defaults).
- **Feature**: An input variable used to make a prediction (e.g., borrower income, loan amount).
- **Train / Val / Test**: Data splits used for building the model (Train), tuning it (Val), and final verification (Test).
- **Pipeline**: A sequence of data processing steps and models chained together.
- **One-Hot Encoding**: A method to convert categorical text data (like "Home Ownership") into numbers.
- **Imputation**: The process of filling in missing data values with estimates (like the median).
- **AUC (Area Under the Curve)**: A metric measuring how well the model distinguishes between "default" and "non-default" (higher is better).
- **RMSE (Root Mean Square Error)**: A measure of the average difference between predicted probabilities and actual outcomes (lower is better).
- **Threshold**: The "cut-off" probability used to decide if a loan should be approved or rejected.
- **Confusion Matrix**: A table showing how many predictions were correct vs. incorrect for each class.
- **Expected Value**: The average outcome of a decision when considering both the probability and the financial impact of different results.


# Project Map / Rubric Map

- EDA (4 variables + business interpretation) → `# EDA (Core)` and `## EDA (Extension)`
- Feature Engineering (≥4 features + economic rationale) → `## Feature Engineering (Economic Intuition)`
- Train/Validation/Test usage (no leakage) → `## Train/Validation/Test Strategy (No Leakage)`
- Models 0–3 + feature importance → `## Models (0–3)` and sub‑sections for each model
- Model comparison → `## Model Comparison`
- Threshold optimization (business) → `## Threshold Optimization (Business)`
- Final test evaluation → `## Final Test Evaluation`
- Conclusion → `## Conclusion`

How to run: Restart Kernel & Run All.


## Rubric Compliance Map

- EDA (4 variables + interpretation) → `## EDA (Core)` and `## EDA (Extension)`
- Feature Engineering (≥4 features + rationale) → `## Feature Engineering (Economic Intuition)`
- Train/Validation/Test protocol → `## Train/Validation/Test Strategy (No Leakage)`
- Models & tuning → `## Models (0–3)` and model subsections
- Model comparison → `## Model Comparison` + `outputs/model_comparison_validation.csv`
- Threshold optimization → `## Threshold Optimization (Business)` + `outputs/threshold_sweep_validation.csv`
- Business decisioning artifacts → `outputs/ev_curve_validation.png`, `outputs/approval_rate_curve_validation.png`
- Confusion matrices → `outputs/confusion_matrix_validation_050.csv`, `outputs/confusion_matrix_validation_optimal.csv`
- Model card → `outputs/model_card.md`


## Objective & Dataset

This notebook builds a supervised credit risk model to estimate default probability from historical loan data, following the course workflow and rubric.


## Rubric Alignment (What is graded)

This notebook follows the rubric outlined in the assignment PDF.

- **Rubric Models (graded)**: Random Forest, Gradient Boosting
- **Optional Extensions (not required)**: Decision Tree, Logistic Regression (baseline)
- **No Test Leakage**: The test set is used only once at the end for a final holdout evaluation.


In [1]:
# Goal: Configure project-wide constants, paths, and payoff assumptions.
# Inputs: Hardcoded strings and dictionary.
# Outputs: Path objects, configuration variables, and PAYOFF dictionary.
# Why it matters: Centralizing configuration ensures consistency across the notebook and defines the business logic for threshold optimization.

# Config
from pathlib import Path

RANDOM_STATE = 42
TARGET_COL = "default"
ID_COL = "id"
TRAIN_PATH = "../data/lending_club_train.csv"
TEST_PATH = "../data/lending_club_test.csv"

REPO_ROOT = Path.cwd() if (Path.cwd() / "data").exists() else Path.cwd().parent
OUTPUT_DIR = REPO_ROOT / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
assert (REPO_ROOT / "data" / "lending_club_train.csv").exists()

PAYOFF = {
    "approve_nondefault": 1,
    "approve_default": -5,
    "reject_default": 0,
    "reject_nondefault": -1,
}


In [2]:
# Goal: Import necessary libraries and set display options.
# Inputs: None.
# Outputs: Loaded Python modules and updated pandas display settings.
# Why it matters: This cell provides all the computational tools required for data processing, visualization, and modeling.

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import warnings
warnings.filterwarnings("ignore", message="FigureCanvasAgg is non-interactive.*")
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, confusion_matrix

pd.set_option("display.max_columns", 200)


## Data Loading


In [3]:
# Goal: Load the training and testing datasets from CSV files.
# Inputs: TRAIN_PATH and TEST_PATH strings.
# Outputs: df_train and df_test pandas DataFrames.
# Why it matters: Loading the raw data is the first step in the machine learning pipeline, enabling all subsequent analysis and modeling.

df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

df_train.shape, df_test.shape


((10129, 21), (2533, 21))

## Data Quality & Target Balance


In [4]:
# Goal: Verify target column existence and check the balance of default vs. non-default cases.
# Inputs: df_train DataFrame and TARGET_COL.
# Outputs: Normalized value counts of the target variable.
# Why it matters: Understanding the class distribution is critical for model evaluation, as imbalanced data can lead to biased model performance.

assert TARGET_COL in df_train.columns
assert set(np.unique(df_train[TARGET_COL])).issubset({0, 1})

df_train[TARGET_COL].value_counts(normalize=True)


default
0    0.807187
1    0.192813
Name: proportion, dtype: float64

## Variable Types

| Variable name | Variable type | Why it matters for credit risk |
| --- | --- | --- |
| int_rate | numerical (continuous) | Higher rates often reflect higher borrower risk and expected loss. |
| dti | numerical (continuous) | Higher debt-to-income suggests tighter cash flow and default risk. |
| loan_amnt | numerical (continuous) | Larger loans increase exposure and repayment burden. |
| grade | categorical (ordinal) | Encodes lender risk grading tied to expected default levels. |


## EDA (Core)


In [5]:
plt.figure(figsize=(6,4))
sns.histplot(df_train["int_rate"], bins=30, kde=True)
plt.title("Distribution of Interest Rates")
plt.xlabel("Interest Rate")
plt.ylabel("Count")
plt.show()


In [6]:
# Goal: Compare interest rates between repaid and defaulted loans.
# Inputs: df_train DataFrame with "int_rate" and "default" columns.
# Outputs: A boxplot comparing interest rate distributions by default status.
# Why it matters: This visualization confirms if higher interest rates are associated with higher default risk, validating its use as a feature.

plt.figure(figsize=(6,4))
sns.boxplot(x="default", y="int_rate", data=df_train)
plt.title("Interest Rate vs Default")
plt.xlabel("Default (0 = Repaid, 1 = Default)")
plt.ylabel("Interest Rate")
plt.show()


**Interest Rate (`int_rate`)**

The interest rate reflects the lender’s assessment of borrower risk at origination.
The distribution shows a wide range of rates, with higher interest rates being more frequent among loans that eventually defaulted.
This suggests that interest rates already embed risk information and are strongly associated with default probability.


In [7]:
# Goal: Visualize the distribution of Debt-to-Income (DTI) ratios.
# Inputs: df_train["dti"].
# Outputs: A histogram plot of DTI ratios.
# Why it matters: High DTI is a classic indicator of financial stress and increased default probability.

plt.figure(figsize=(6,4))
sns.histplot(df_train["dti"], bins=30, kde=True)
plt.title("Distribution of Debt-to-Income Ratio (DTI)")
plt.xlabel("DTI")
plt.ylabel("Count")
plt.show()


In [8]:
# Goal: Compare DTI distributions between repaid and defaulted loans.
# Inputs: df_train DataFrame with "dti" and "default" columns.
# Outputs: A boxplot comparing DTI distributions by default status.
# Why it matters: Identifying a higher DTI for defaulted loans supports the economic rationale for its inclusion in the model.

plt.figure(figsize=(6,4))
sns.boxplot(x="default", y="dti", data=df_train)
plt.title("DTI vs Default")
plt.xlabel("Default (0 = Repaid, 1 = Default)")
plt.ylabel("Debt-to-Income Ratio")
plt.show()


**Debt-to-Income Ratio (`dti`)**

The debt-to-income ratio measures the proportion of a borrower’s income already committed to debt obligations.
Higher DTI values indicate greater financial strain and reduced repayment capacity.
The analysis shows that defaulted loans tend to have higher DTI levels, confirming DTI as a key risk driver in credit decisions.


In [9]:
# Goal: Visualize the distribution of loan amounts.
# Inputs: df_train["loan_amnt"].
# Outputs: A histogram plot of loan amounts.
# Why it matters: Understanding the scale of loans helps assess the overall risk exposure.

plt.figure(figsize=(6,4))
sns.histplot(df_train["loan_amnt"], bins=30, kde=True)
plt.title("Distribution of Loan Amount")
plt.xlabel("Loan Amount")
plt.ylabel("Count")
plt.show()


In [10]:
# Goal: Compare loan amount distributions by default status.
# Inputs: df_train DataFrame with "loan_amnt" and "default" columns.
# Outputs: A boxplot comparing loan amount distributions.
# Why it matters: While loan size impacts loss severity, its relationship with default probability might be less direct than other factors.

plt.figure(figsize=(6,4))
sns.boxplot(x="default", y="loan_amnt", data=df_train)
plt.title("Loan Amount vs Default")
plt.xlabel("Default (0 = Repaid, 1 = Default)")
plt.ylabel("Loan Amount")
plt.show()


**Loan Amount (`loan_amnt`)**

The loan amount represents the size of the lender’s exposure rather than the borrower’s intrinsic credit quality.
The distribution shows a concentration around mid-sized loans.
The relationship with default appears weaker than for interest rate or DTI, suggesting that loan size alone is not a primary driver of default, but may interact with income or other risk factors.


In [11]:
# Goal: Show the frequency of different credit grades in the data.
# Inputs: df_train["grade"].
# Outputs: A count plot of borrowers per credit grade.
# Why it matters: Credit grade is a synthesized risk measure provided by the lender; understanding its distribution is key for assessment.

plt.figure(figsize=(6,4))
sns.countplot(x="grade", data=df_train, order=sorted(df_train["grade"].unique()))
plt.title("Distribution of Credit Grades")
plt.xlabel("Credit Grade")
plt.ylabel("Count")
plt.show()


In [12]:
# Goal: Calculate and plot the average default rate for each credit grade.
# Inputs: df_train DataFrame grouped by "grade".
# Outputs: A bar plot of default rates by grade.
# Why it matters: This highlights the strong monotonic relationship between internal risk grades and actual default outcomes.

grade_default_rate = (
    df_train
    .groupby("grade")["default"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(6,4))
sns.barplot(x="grade", y="default", data=grade_default_rate,
            order=sorted(grade_default_rate["grade"]))
plt.title("Average Default Rate by Credit Grade")
plt.xlabel("Credit Grade")
plt.ylabel("Default Rate")
plt.show()


**Credit Grade (`grade`)**

The credit grade is an ordinal categorical variable summarizing the lender’s internal credit assessment at origination.
Lower grades are associated with significantly higher default rates, while higher grades exhibit substantially lower default frequencies.
This clear monotonic relationship confirms that `grade` is one of the strongest predictors of default and validates its economic relevance in credit risk modeling.


## EDA (Extension)


In [13]:
# Goal: Identify and list all numerical columns for correlation analysis.
# Inputs: df_train DataFrame.
# Outputs: A list of numerical column names (num_cols).
# Why it matters: Automated column selection ensures all relevant numeric features are included in the correlation analysis.

# Select numerical columns only
num_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()

# Exclude target itself
num_cols = [c for c in num_cols if c != "default"]

num_cols


['id',
 'loan_amnt',
 'term',
 'int_rate',
 'installment',
 'annual_inc',
 'emp_length',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc']

In [14]:
# Goal: Calculate the linear correlation between each numerical feature and the default target.
# Inputs: df_train numerical columns.
# Outputs: A sorted list of correlations with the default indicator.
# Why it matters: This identifies which individual variables have the strongest linear relationship with loan default.

corr_with_target = (
    df_train[num_cols + ["default"]]
    .corr()["default"]
    .drop("default")
    .sort_values(ascending=False)
)

corr_with_target.to_frame(name="Correlation with Default")


,Correlation with Default
int_rate,0.251016
term,0.169246
dti,0.102043
inq_last_6mths,0.073969
revol_util,0.072172
loan_amnt,0.068005
installment,0.054795
id,0.042234
pub_rec,0.037639
delinq_2yrs,0.017844


## Feature Engineering (Economic Intuition)

### Feature Engineering Strategy

Feature engineering was guided by economic intuition rather than brute-force expansion. Engineered variables capture distinct borrower-risk dimensions: affordability/burden, leverage/utilization, credit experience/stability, and nonlinear effects, while maintaining interpretability and model stability.

| Feature | Intuition |
| --- | --- |
| installment_to_income | installment / annual_inc |
| loan_to_income | loan_amnt / annual_inc |
| revol_balance_util | revol_bal * revol_util |
| revol_balance_to_income | revol_bal / annual_inc |
| open_to_total_acc | open_acc / total_acc |
| recent_inquiry_flag | (inq_last_6mths > 0) |
| log_annual_inc | log(annual_inc) |
| sqrt_dti | sqrt(dti) |


In [15]:
# Goal: Prepare data by separating the target variable and dropping identifiers.
# Inputs: df_train DataFrame.
# Outputs: X (features) and y (target).
# Why it matters: Proper separation of inputs and outcomes is essential for the Scikit-Learn training process.

ID_COL = "id" if "id" in df_train.columns else None

y = df_train[TARGET_COL].astype(int)
X = df_train.drop(columns=[TARGET_COL] + ([ID_COL] if ID_COL else []))

if ID_COL is not None:
    assert ID_COL not in X.columns


In [16]:
def add_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    annual_inc_safe = df["annual_inc"].clip(lower=1)
    total_acc_safe = df["total_acc"].replace(0, np.nan)

    df["installment_to_income"] = df["installment"] / annual_inc_safe
    df["loan_to_income"] = df["loan_amnt"] / annual_inc_safe
    df["revol_balance_util"] = df["revol_bal"] * df["revol_util"]
    df["revol_balance_to_income"] = df["revol_bal"] / annual_inc_safe
    df["open_to_total_acc"] = df["open_acc"] / total_acc_safe
    df["recent_inquiry_flag"] = (df["inq_last_6mths"] > 0).astype(int)
    df["log_annual_inc"] = np.log(annual_inc_safe)
    df["sqrt_dti"] = np.sqrt(df["dti"].clip(lower=0))

    return df

engineered_cols = [
    "installment_to_income",
    "loan_to_income",
    "revol_balance_util",
    "revol_balance_to_income",
    "open_to_total_acc",
    "recent_inquiry_flag",
    "log_annual_inc",
    "sqrt_dti",
]

X_fe = add_engineered_features(X)


In [17]:
# Goal: Inspect the statistics and missing values of the newly engineered features.
# Inputs: X_fe engineered columns.
# Outputs: Summary statistics (mean, std, min, max) and missing value percentages.
# Why it matters: Verification ensure the features were created correctly and don't introduce excessive missing data.

print("Engineered columns:", engineered_cols)

X_fe[engineered_cols].describe().T[["mean", "std", "min", "max"]]

X_fe[engineered_cols].isna().mean().sort_values(ascending=False)


Engineered columns: ['installment_to_income', 'loan_to_income', 'revol_balance_util', 'revol_balance_to_income', 'open_to_total_acc', 'recent_inquiry_flag', 'log_annual_inc', 'sqrt_dti']


installment_to_income      0.0
loan_to_income             0.0
revol_balance_util         0.0
revol_balance_to_income    0.0
open_to_total_acc          0.0
recent_inquiry_flag        0.0
log_annual_inc             0.0
sqrt_dti                   0.0
dtype: float64

In [18]:
# Goal: Visualize the correlation matrix of numerical variables using a heatmap.
# Inputs: df_train numerical columns and default indicator.
# Outputs: A heatmap showing correlation coefficients between all numeric variables.
# Why it matters: Heatmaps reveal multicollinearity (highly correlated inputs) and help identify which features are most related to the target.

plt.figure(figsize=(8,6))
sns.heatmap(
    df_train[num_cols + ["default"]].corr(),
    cmap="coolwarm",
    center=0,
    linewidths=0.5
)
plt.title("Correlation Matrix (Numerical Variables)")
plt.show()


**Correlation with Default (Numerical Variables)**

As an extension of the initial EDA, we examine linear correlations between numerical variables and the default indicator.
Variables such as interest rate and debt-to-income ratio show stronger positive correlations with default, while income-related variables tend to be negatively correlated.
These results are consistent with economic intuition and support the relevance of these variables for credit risk modeling.


In [19]:
# Goal: Segment the DTI variable into quartiles to prepare for non-linear analysis.
# Inputs: df_train["dti"].
# Outputs: A new temporary column "dti_quartile" in df_train.
# Why it matters: Discretizing continuous variables can sometimes reveal non-linear patterns that a simple correlation might miss.

df_train["dti_quartile"] = pd.qcut(df_train["dti"], q=4, labels=["Q1 (Low)", "Q2", "Q3", "Q4 (High)"])


In [20]:
dti_default_by_quantile = (
    df_train
    .groupby("dti_quartile", observed=False)["default"]
    .mean()
    .reset_index()
)

dti_default_by_quantile


,dti_quartile,default
0,Q1 (Low),0.151599
1,Q2,0.161672
2,Q3,0.212732
3,Q4 (High),0.245358


In [21]:
# Goal: Plot the default rate across DTI quartiles.
# Inputs: dti_default_by_quantile summary table.
# Outputs: A bar plot showing default rate by DTI segment.
# Why it matters: Visualizing the trend across quantiles provides a clear, non-technical proof of the risk associated with high DTI.

plt.figure(figsize=(6,4))
sns.barplot(
    x="dti_quartile",
    y="default",
    data=dti_default_by_quantile
)
plt.title("Default Rate by DTI Quartile")
plt.xlabel("DTI Quartile")
plt.ylabel("Average Default Rate")
plt.show()


**Default Rate by DTI Quantiles**

To capture potential nonlinear effects, we analyze default rates across quartiles of the debt-to-income ratio.
Default probability increases monotonically from the lowest to the highest DTI quartile, indicating that borrower risk rises disproportionately at higher levels of indebtedness.
This supports the inclusion of nonlinear transformations or interaction-based features in subsequent modeling steps.


In [22]:
df_train.drop(columns=["dti_quartile"], inplace=True)


In [23]:
# Goal: Quickly inspect the first few rows of the training data.
# Inputs: df_train DataFrame.
# Outputs: Display of the top 5 rows.
# Why it matters: A quick visual check ensures data loading and basic transformations were successful.

df_train.head()


,id,loan_amnt,term,int_rate,installment,grade,sub_grade,annual_inc,emp_length,home_ownership,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,purpose,default
0,79044496,16000.0,60.0,12.99,363.97,C,C2,49000.0,5.0,RENT,Source Verified,15.94,0.0,0.0,9.0,0.0,26376.0,85.4,19.0,debt_consolidation,0
1,43246030,15150.0,36.0,10.99,495.92,B,B4,38000.0,8.0,MORTGAGE,Not Verified,39.96,1.0,1.0,10.0,0.0,19459.0,63.2,38.0,credit_card,0
2,641694,12800.0,60.0,14.83,303.38,D,D3,75000.0,0.0,RENT,Source Verified,12.43,0.0,0.0,12.0,0.0,8078.0,35.0,13.0,moving,0
3,70981628,6000.0,36.0,9.75,192.90,B,B3,70000.0,0.0,MORTGAGE,Verified,25.53,1.0,1.0,9.0,0.0,2048.0,39.4,16.0,debt_consolidation,0
4,57792301,15000.0,60.0,13.33,343.84,C,C3,96000.0,3.0,RENT,Source Verified,13.44,0.0,0.0,11.0,1.0,15979.0,53.3,36.0,debt_consolidation,0


In [24]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10129 non-null  int64  
 1   loan_amnt            10129 non-null  float64
 2   term                 10129 non-null  float64
 3   int_rate             10129 non-null  float64
 4   installment          10129 non-null  float64
 5   grade                10129 non-null  object 
 6   sub_grade            10129 non-null  object 
 7   annual_inc           10129 non-null  float64
 8   emp_length           10129 non-null  float64
 9   home_ownership       10129 non-null  object 
 10  verification_status  10129 non-null  object 
 11  dti                  10129 non-null  float64
 12  delinq_2yrs          10129 non-null  float64
 13  inq_last_6mths       10129 non-null  float64
 14  open_acc             10129 non-null  float64
 15  pub_rec              10129 non-null 

In [25]:
# Goal: List the data types for all columns in the training set.
# Inputs: df_train DataFrame.
# Outputs: Series of column names and their associated data types.
# Why it matters: Ensuring columns have correct types (e.g., numeric vs. categorical) is vital for proper preprocessing.

df_train.dtypes


id                       int64
loan_amnt              float64
term                   float64
int_rate               float64
installment            float64
grade                   object
sub_grade               object
annual_inc             float64
emp_length             float64
home_ownership          object
verification_status     object
dti                    float64
delinq_2yrs            float64
inq_last_6mths         float64
open_acc               float64
pub_rec                float64
revol_bal              float64
revol_util             float64
total_acc              float64
purpose                 object
default                  int64
dtype: object

## Train/Validation/Test Strategy (No Leakage)

Training uses only `X_train`/`y_train`, validation is used for tuning, and the test set is held out for a single final evaluation at the end of the notebook. The test set is not used for model selection, tuning, feature selection, or threshold optimization.


## Models (0–3)


Tuning grids are intentionally small to ensure fast execution and reproducible results.


## Model 0 — Logistic Regression (Baseline)

This baseline provides a simple, interpretable benchmark for credit risk prediction. Logistic Regression serves as a linear reference point for comparison with more complex models.


In [26]:
X_train, X_val, y_train, y_val = train_test_split(
    X_fe,
    y,
    test_size=0.25,
    stratify=y,
    random_state=RANDOM_STATE,
)


In [27]:
# Goal: Confirm the train/validation split sizes and target prevalence.
# Inputs: X_train, X_val, y_train, y_val.
# Outputs: Printed summary statistics.
# Why it matters: Verifies the split is stable and representative before modeling.

print(f"Train rows: {X_train.shape[0]}, Validation rows: {X_val.shape[0]}")
print(f"Train default rate: {y_train.mean():.4f}, Validation default rate: {y_val.mean():.4f}")


Train rows: 7596, Validation rows: 2533
Train default rate: 0.1929, Validation default rate: 0.1927


In [28]:
# Goal: Build a systematic pipeline to handle missing values and scale/encode different data types.
# Inputs: X_train column definitions.
# Outputs: A "preprocess" ColumnTransformer object.
# Why it matters: Standardizing data ensures that the machine learning models treat all features fairly and can handle real-world data issues like missing values.

categorical_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X_train.columns if c not in categorical_cols]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)


In [29]:
log_reg = LogisticRegression(
    solver="lbfgs",
    max_iter=1000,
    random_state=RANDOM_STATE,
)

log_reg_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", log_reg),
    ]
)

log_reg_pipeline.fit(X_train, y_train)
print("Pipeline fitted. Preprocess: num + cat. Model: LogisticRegression.")

Pipeline fitted. Preprocess: num + cat. Model: LogisticRegression.


In [30]:
# Goal: Define a helper function to calculate key metrics (AUC, RMSE) for both training and validation sets.
# Inputs: Trained pipeline and datasets.
# Outputs: A dictionary containing performance metrics and the "results_df" placeholder.
# Why it matters: This function ensures that all models are evaluated consistently, making their results directly comparable.

def evaluate_model(model_name, pipeline, X_train, y_train, X_val, y_val):
    train_probs = pipeline.predict_proba(X_train)[:, 1]
    val_probs = pipeline.predict_proba(X_val)[:, 1]

    train_rmse = np.sqrt(mean_squared_error(y_train, train_probs))
    val_rmse = np.sqrt(mean_squared_error(y_val, val_probs))

    train_auc = roc_auc_score(y_train, train_probs)
    val_auc = roc_auc_score(y_val, val_probs)

    return {
        "Model": model_name,
        "Train_RMSE": train_rmse,
        "Val_RMSE": val_rmse,
        "Train_AUC": train_auc,
        "Val_AUC": val_auc,
        "AUC_Gap": train_auc - val_auc,
        "RMSE_Gap": val_rmse - train_rmse,
    }

results_df = pd.DataFrame()


In [31]:
# Goal: Evaluate the Logistic Regression model and store its metrics.
# Inputs: log_reg_pipeline and datasets.
# Outputs: Updated results_df with Logistic Regression metrics.
# Why it matters: This establishes the first entry in our model comparison table.

log_reg_metrics = evaluate_model(
    "Logistic Regression",
    log_reg_pipeline,
    X_train,
    y_train,
    X_val,
    y_val,
)

results_df = pd.concat([results_df, pd.DataFrame([log_reg_metrics])], ignore_index=True)

results_df


,Model,Train_RMSE,Val_RMSE,Train_AUC,Val_AUC,AUC_Gap,RMSE_Gap
0,Logistic Regression,0.374962,0.379806,0.712319,0.690397,0.021921,0.004845


## Model 1 — Decision Tree (Tuned)

Decision Trees capture nonlinearities and interactions among borrower characteristics. Because they can overfit, we tune key hyperparameters using the validation set.


In [32]:
# Goal: Set up a dedicated preprocessing pipeline for tree-based models (no scaling needed).
# Inputs: X_train column definitions.
# Outputs: A "preprocess_tree" ColumnTransformer object.
# Why it matters: Tree-based models are naturally scale-invariant, so we only need to handle missing values and encoding.

categorical_cols_tree = X_train.select_dtypes(include=["object"]).columns.tolist()
num_cols_tree = [c for c in X_train.columns if c not in categorical_cols_tree]

numeric_transformer_tree = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_transformer_tree = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess_tree = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_tree, num_cols_tree),
        ("cat", categorical_transformer_tree, categorical_cols_tree),
    ]
)


In [33]:
# Goal: Optimize Decision Tree hyperparameters by testing different depth and leaf size combinations.
# Inputs: Param grid for max_depth and min_samples_leaf.
# Outputs: dt_tuning_df containing results for all combinations.
# Why it matters: Tuning prevents the tree from becoming too complex (overfitting) or too simple (underfitting).

param_grid = {
    "max_depth": [3, 5, 7, 10],
    "min_samples_leaf": [50, 100, 200],
}

dt_tuning_results = []

for max_depth in param_grid["max_depth"]:
    for min_samples_leaf in param_grid["min_samples_leaf"]:
        tree_model = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            random_state=RANDOM_STATE,
        )

        tree_pipeline = Pipeline(
            steps=[
                ("preprocess", preprocess_tree),
                ("model", tree_model),
            ]
        )

        tree_pipeline.fit(X_train, y_train)
        train_probs = tree_pipeline.predict_proba(X_train)[:, 1]
        val_probs = tree_pipeline.predict_proba(X_val)[:, 1]

        dt_tuning_results.append(
            {
                "max_depth": max_depth,
                "min_samples_leaf": min_samples_leaf,
                "train_auc": roc_auc_score(y_train, train_probs),
                "val_auc": roc_auc_score(y_val, val_probs),
                "chosen_flag": False,
            }
        )

dt_tuning_df = pd.DataFrame(dt_tuning_results)

best_dt_idx = dt_tuning_df["val_auc"].idxmax()
dt_tuning_df.loc[best_dt_idx, "chosen_flag"] = True
best_dt_params = dt_tuning_df.loc[best_dt_idx, ["max_depth", "min_samples_leaf"]].to_dict()

# Keep compatibility with prior naming
best_params = best_dt_params
tuning_df = dt_tuning_df

dt_tuning_df.sort_values("val_auc", ascending=False).head(10)


,max_depth,min_samples_leaf,train_auc,val_auc,chosen_flag
0,3,50,0.689931,0.667967,True
1,3,100,0.689931,0.667967,False
2,3,200,0.689931,0.667967,False
11,10,200,0.715711,0.659281,False
8,7,200,0.715711,0.659281,False
3,5,50,0.719240,0.656571,False
5,5,200,0.712199,0.656276,False
4,5,100,0.720761,0.654881,False
7,7,100,0.736669,0.648326,False
10,10,100,0.741372,0.646720,False


In [34]:
# Goal: Train the Decision Tree model using the best identified parameters.
# Inputs: Optimal max_depth and min_samples_leaf.
# Outputs: A fitted best_tree_pipeline object.
# Why it matters: This completes the training of our second model candidate using an optimized configuration.

best_params = tuning_df.loc[tuning_df["val_auc"].idxmax()].to_dict()

best_tree = DecisionTreeClassifier(
    max_depth=int(best_params["max_depth"]),
    min_samples_leaf=int(best_params["min_samples_leaf"]),
    random_state=RANDOM_STATE,
)

best_tree_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_tree),
        ("model", best_tree),
    ]
)

best_tree_pipeline.fit(X_train, y_train)
print("Pipeline fitted. Preprocess: num + cat. Model: DecisionTreeClassifier.")


Pipeline fitted. Preprocess: num + cat. Model: DecisionTreeClassifier.


In [35]:
# Goal: Evaluate the optimized Decision Tree and add its metrics to the comparison table.
# Inputs: best_tree_pipeline and datasets.
# Outputs: Updated results_df with Decision Tree metrics.
# Why it matters: This allows us to see if the Decision Tree outperforms the Logistic Regression baseline.

dt_metrics = evaluate_model(
    "Decision Tree",
    best_tree_pipeline,
    X_train,
    y_train,
    X_val,
    y_val,
)

results_df = pd.concat([results_df, pd.DataFrame([dt_metrics])], ignore_index=True)

results_df


,Model,Train_RMSE,Val_RMSE,Train_AUC,Val_AUC,AUC_Gap,RMSE_Gap
0,Logistic Regression,0.374962,0.379806,0.712319,0.690397,0.021921,0.004845
1,Decision Tree,0.379684,0.383817,0.689931,0.667967,0.021965,0.004133


In [36]:
feature_names = best_tree_pipeline.named_steps["preprocess"].get_feature_names_out()
importances = best_tree_pipeline.named_steps["model"].feature_importances_

importance_df = pd.DataFrame(
    {"feature": feature_names, "importance": importances}
).sort_values("importance", ascending=False)

plt.figure(figsize=(8, 4))
sns.barplot(
    x="importance",
    y="feature",
    data=importance_df.head(10),
    orient="h",
)
plt.title("Top 10 Feature Importances (Decision Tree)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


## Model 2 — Random Forest (Bagging)

Random Forest reduces variance via bagging and feature subsampling.


In [37]:
# Goal: Tune and train a Random Forest model, which uses an ensemble of trees to improve performance.
# Inputs: X_train and y_train datasets, RF hyperparameter distributions.
# Outputs: A fitted rf_best_pipeline, rf_tuning_df, and updated results_df.
# Why it matters: Random Forests are often more robust and accurate than single trees because they average out individual errors.

categorical_cols_rf = X_train.select_dtypes(include=["object"]).columns.tolist()
num_cols_rf = [c for c in X_train.columns if c not in categorical_cols_rf]

numeric_transformer_rf = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_transformer_rf = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess_rf = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_rf, num_cols_rf),
        ("cat", categorical_transformer_rf, categorical_cols_rf),
    ]
)

rf_base_model = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1,
    bootstrap=True,
)

rf_param_distributions = {
    "model__n_estimators": [300, 600, 1000],
    "model__max_features": ["sqrt", 0.5, 0.8],
    "model__max_depth": [None, 6, 10, 14, 18],
    "model__min_samples_split": [2, 10, 50],
    "model__min_samples_leaf": [1, 5, 20, 50],
    "model__class_weight": [None, "balanced"],
    "model__bootstrap": [True],
}

rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_rf),
        ("model", rf_base_model),
    ]
)

rf_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=rf_param_distributions,
    n_iter=60,
    scoring="roc_auc",
    cv=3,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

rf_search.fit(X_train, y_train)

print("Sanity check: df_test not used in Random Forest tuning (train/val only).")
print(f"RF best CV AUC: {rf_search.best_score_:.6f}")
print("RF best params:", rf_search.best_params_)

rf_cv_results = pd.DataFrame(rf_search.cv_results_)
rf_tuning_df = rf_cv_results[
    [
        "param_model__n_estimators",
        "param_model__max_features",
        "param_model__max_depth",
        "param_model__min_samples_split",
        "param_model__min_samples_leaf",
        "param_model__class_weight",
        "param_model__bootstrap",
        "mean_test_score",
    ]
].copy()

rf_tuning_df = rf_tuning_df.rename(
    columns={
        "param_model__n_estimators": "n_estimators",
        "param_model__max_features": "max_features",
        "param_model__max_depth": "max_depth",
        "param_model__min_samples_split": "min_samples_split",
        "param_model__min_samples_leaf": "min_samples_leaf",
        "param_model__class_weight": "class_weight",
        "param_model__bootstrap": "bootstrap",
        "mean_test_score": "val_auc",
    }
)

rf_tuning_df["chosen_flag"] = False
best_rf_idx = rf_tuning_df["val_auc"].astype(float).idxmax()
rf_tuning_df.loc[best_rf_idx, "chosen_flag"] = True

rf_tuning_df.sort_values("val_auc", ascending=False).head(10)

rf_best_pipeline = rf_search.best_estimator_
print("Pipeline fitted. Preprocess: num + cat. Model: RandomForestClassifier.")

rf_metrics = evaluate_model(
    "Random Forest",
    rf_best_pipeline,
    X_train,
    y_train,
    X_val,
    y_val,
)

print(f"RF train vs val AUC gap: {rf_metrics['AUC_Gap']:.6f}")

results_df = pd.concat([results_df, pd.DataFrame([rf_metrics])], ignore_index=True)

results_df


C:\Users\mail\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Sanity check: df_test not used in Random Forest tuning (train/val only).
RF best CV AUC: 0.694991
RF best params: {'model__n_estimators': 600, 'model__min_samples_split': 10, 'model__min_samples_leaf': 50, 'model__max_features': 'sqrt', 'model__max_depth': 10, 'model__class_weight': 'balanced', 'model__bootstrap': True}
Pipeline fitted. Preprocess: num + cat. Model: RandomForestClassifier.


RF train vs val AUC gap: 0.077668


,Model,Train_RMSE,Val_RMSE,Train_AUC,Val_AUC,AUC_Gap,RMSE_Gap
0,Logistic Regression,0.374962,0.379806,0.712319,0.690397,0.021921,0.004845
1,Decision Tree,0.379684,0.383817,0.689931,0.667967,0.021965,0.004133
2,Random Forest,0.445107,0.462537,0.773063,0.695395,0.077668,0.017429


In [38]:
rf_feature_names = rf_best_pipeline.named_steps["preprocess"].get_feature_names_out()
rf_importances = rf_best_pipeline.named_steps["model"].feature_importances_

rf_importance_df = pd.DataFrame(
    {"feature": rf_feature_names, "importance": rf_importances}
).sort_values("importance", ascending=False)

plt.figure(figsize=(8, 4))
sns.barplot(
    x="importance",
    y="feature",
    data=rf_importance_df.head(10),
    orient="h",
)
plt.title("Top 10 Feature Importances (Random Forest)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


### Interpreting Feature Importance

The chart above shows which variables the model relies on most to predict loan defaults. Here is a plain-English interpretation of the top features:

1.  **Interest Rate (`int_rate`)**: Borrowers with higher interest rates are more likely to default. This is often because the lender already identified them as higher risk at the time of the loan application.
2.  **Credit Grade (`grade_...`)**: Internal risk grades assigned by the lender are strong predictors. Lower grades (e.g., E, F, G) typically show a higher risk of default.
3.  **Debt-to-Income Ratio (`dti` or `sqrt_dti`)**: Borrowers who spend a larger portion of their income on debt payments are more likely to struggle with new loan repayments.
4.  **Revolving Utilization (`revol_util`)**: Using a high percentage of available credit card limits suggests a borrower may be overextended.
5.  **Inquiries in Last 6 Months (`inq_last_6mths`)**: A high number of recent credit applications can be a sign of financial distress or an urgent need for cash.
6.  **Annual Income (`annual_inc` or `log_annual_inc`)**: Higher income generally correlates with a lower risk of default, as borrowers have more cushion to handle unexpected expenses.

**Note**: These are *correlational* signals, not *causal*. For example, a high interest rate doesn't necessarily *cause* a default; rather, high-risk borrowers are both more likely to default and more likely to be charged higher rates.


## Model 3 — Gradient Boosting (Boosting)

Gradient Boosting reduces bias via sequential weak learners.


In [39]:
# Goal: Tune and train a Gradient Boosting model to capture complex, non-linear patterns.
# Inputs: X_train and y_train datasets, GB hyperparameter distributions.
# Outputs: A fitted gb_best_pipeline, gb_tuning_df, and updated results_df.
# Why it matters: Gradient Boosting is a powerful ensemble method that often provides the highest accuracy in credit risk tasks.

categorical_cols_gb = X_train.select_dtypes(include=["object"]).columns.tolist()
num_cols_gb = [c for c in X_train.columns if c not in categorical_cols_gb]

numeric_transformer_gb = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_transformer_gb = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess_gb = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_gb, num_cols_gb),
        ("cat", categorical_transformer_gb, categorical_cols_gb),
    ]
)

gb_base_model = GradientBoostingClassifier(
    random_state=RANDOM_STATE,
)

gb_param_distributions = {
    "model__n_estimators": [100, 300, 600],
    "model__learning_rate": [0.03, 0.05, 0.1],
    "model__subsample": [0.6, 0.8, 1.0],
    "model__max_depth": [2, 3],
    "model__min_samples_leaf": [20, 50, 100],
    "model__min_samples_split": [2, 10, 50],
    "model__max_features": [None, "sqrt"],
}

gb_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_gb),
        ("model", gb_base_model),
    ]
)

gb_search = RandomizedSearchCV(
    gb_pipeline,
    param_distributions=gb_param_distributions,
    n_iter=60,
    scoring="roc_auc",
    cv=3,
    random_state=RANDOM_STATE,
)

gb_search.fit(X_train, y_train)

print("Sanity check: df_test not used in Gradient Boosting tuning (train/val only).")
print(f"GB best CV AUC: {gb_search.best_score_:.6f}")
print("GB best params:", gb_search.best_params_)

gb_cv_results = pd.DataFrame(gb_search.cv_results_)
gb_tuning_df = gb_cv_results[
    [
        "param_model__n_estimators",
        "param_model__learning_rate",
        "param_model__subsample",
        "param_model__max_depth",
        "param_model__min_samples_leaf",
        "param_model__min_samples_split",
        "param_model__max_features",
        "mean_test_score",
    ]
].copy()

gb_tuning_df = gb_tuning_df.rename(
    columns={
        "param_model__n_estimators": "n_estimators",
        "param_model__learning_rate": "learning_rate",
        "param_model__subsample": "subsample",
        "param_model__max_depth": "max_depth",
        "param_model__min_samples_leaf": "min_samples_leaf",
        "param_model__min_samples_split": "min_samples_split",
        "param_model__max_features": "max_features",
        "mean_test_score": "val_auc",
    }
)

gb_tuning_df["chosen_flag"] = False
best_gb_idx = gb_tuning_df["val_auc"].astype(float).idxmax()
gb_tuning_df.loc[best_gb_idx, "chosen_flag"] = True

gb_tuning_df.sort_values("val_auc", ascending=False).head(10)

gb_best_pipeline = gb_search.best_estimator_
print("Pipeline fitted. Preprocess: num + cat. Model: GradientBoostingClassifier.")

gb_metrics = evaluate_model(
    "Gradient Boosting",
    gb_best_pipeline,
    X_train,
    y_train,
    X_val,
    y_val,
)

print(f"GB train vs val AUC gap: {gb_metrics['AUC_Gap']:.6f}")

results_df = pd.concat([results_df, pd.DataFrame([gb_metrics])], ignore_index=True)

results_df


Sanity check: df_test not used in Gradient Boosting tuning (train/val only).
GB best CV AUC: 0.696879
GB best params: {'model__subsample': 0.8, 'model__n_estimators': 100, 'model__min_samples_split': 10, 'model__min_samples_leaf': 50, 'model__max_features': 'sqrt', 'model__max_depth': 3, 'model__learning_rate': 0.03}
Pipeline fitted. Preprocess: num + cat. Model: GradientBoostingClassifier.
GB train vs val AUC gap: 0.035039


,Model,Train_RMSE,Val_RMSE,Train_AUC,Val_AUC,AUC_Gap,RMSE_Gap
0,Logistic Regression,0.374962,0.379806,0.712319,0.690397,0.021921,0.004845
1,Decision Tree,0.379684,0.383817,0.689931,0.667967,0.021965,0.004133
2,Random Forest,0.445107,0.462537,0.773063,0.695395,0.077668,0.017429
3,Gradient Boosting,0.375600,0.380497,0.724127,0.689088,0.035039,0.004897


In [40]:
gb_feature_names = gb_best_pipeline.named_steps["preprocess"].get_feature_names_out()
gb_importances = gb_best_pipeline.named_steps["model"].feature_importances_

gb_importance_df = pd.DataFrame(
    {"feature": gb_feature_names, "importance": gb_importances}
).sort_values("importance", ascending=False)

# Goal: Visualize feature importances for the Gradient Boosting model.
# Inputs: gb_best_pipeline model importances.
# Outputs: A horizontal bar plot of top-10 features.
# Why it matters: Comparing feature importance across models helps build a more reliable consensus on which variables truly drive risk.

plt.figure(figsize=(8, 4))
sns.barplot(
    x="importance",
    y="feature",
    data=gb_importance_df.head(10),
    orient="h",
)
plt.title("Top 10 Feature Importances (Gradient Boosting)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


## Model Comparison


In [41]:
# Goal: Aggregate and format metrics from all models for final comparison.
# Inputs: results_df from all model evaluation steps.
# Outputs: A sorted comparison_df showing AUC and RMSE for all candidates.
# Why it matters: This table provides a clear, quantitative basis for choosing the final model to be used in production.

comparison_df = results_df[
    ["Model", "Train_AUC", "Val_AUC", "AUC_Gap", "Train_RMSE", "Val_RMSE", "RMSE_Gap"]
].copy()

rubric_models = ["Random Forest", "Gradient Boosting"]
comparison_df["role"] = comparison_df["Model"].apply(
    lambda m: "Rubric Model" if m in rubric_models else "Optional Extension"
)

comparison_df[["Train_AUC", "Val_AUC", "AUC_Gap", "Train_RMSE", "Val_RMSE", "RMSE_Gap"]] = (
    comparison_df[["Train_AUC", "Val_AUC", "AUC_Gap", "Train_RMSE", "Val_RMSE", "RMSE_Gap"]].round(6)
)

comparison_df = comparison_df[
    ["Model", "role", "Train_AUC", "Val_AUC", "AUC_Gap", "Train_RMSE", "Val_RMSE", "RMSE_Gap"]
]
comparison_df = comparison_df.sort_values("Val_AUC", ascending=False).reset_index(drop=True)
comparison_df


,Model,role,Train_AUC,Val_AUC,AUC_Gap,Train_RMSE,Val_RMSE,RMSE_Gap
0,Random Forest,Rubric Model,0.773063,0.695395,0.077668,0.445107,0.462537,0.017429
1,Logistic Regression,Optional Extension,0.712319,0.690397,0.021921,0.374962,0.379806,0.004845
2,Gradient Boosting,Rubric Model,0.724127,0.689088,0.035039,0.375600,0.380497,0.004897
3,Decision Tree,Optional Extension,0.689931,0.667967,0.021965,0.379684,0.383817,0.004133


The model with the highest validation AUC is selected as the leading candidate. The AUC_Gap column highlights overfitting risk: larger gaps indicate stronger train–validation divergence. RMSE values provide a probability calibration check; small RMSE_Gap values suggest stable generalization. Overall improvements are modest but consistent with credit‑risk modeling norms.


## Feature Ablation: Removing grade/sub_grade

We test whether the internal Lending Club risk labels (`grade`, `sub_grade`) dominate signal. These labels are available at origination, but relying on them can reduce interpretability and may double-count information already reflected in rates and terms.

The table below compares validation performance using the tuned Random Forest with:
- **FeatureSet_A (Full)**: all current features
- **FeatureSet_B (No-Grade)**: drops `grade` and `sub_grade` before preprocessing


In [42]:
# Goal: Compare performance with and without grade/sub_grade using fixed RF params.
# Inputs: X_train, X_val, y_train, y_val.
# Outputs: feature_ablation_df with validation metrics.
# Why it matters: Confirms whether internal grade labels dominate signal or if performance is stable without them.

rf_ablation_params = {
    "n_estimators": 600,
    "max_features": "sqrt",
    "max_depth": 10,
    "min_samples_split": 10,
    "min_samples_leaf": 50,
    "class_weight": "balanced",
    "bootstrap": True,
    "random_state": RANDOM_STATE,
    "n_jobs": -1,
}

print("Sanity check: df_test not used in feature ablation (train/val only).")

drop_cols = [c for c in ["grade", "sub_grade"] if c in X_train.columns]
print(f"Dropping columns for FeatureSet_B: {drop_cols if drop_cols else 'None'}")

X_train_full = X_train.copy()
X_val_full = X_val.copy()

X_train_nograde = X_train_full.drop(columns=drop_cols)
X_val_nograde = X_val_full.drop(columns=drop_cols)


def build_rf_pipeline_for_ablation(train_X: pd.DataFrame) -> Pipeline:
    cat_cols = train_X.select_dtypes(include=["object"]).columns.tolist()
    num_cols = [c for c in train_X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ]
    )

    model = RandomForestClassifier(**rf_ablation_params)

    return Pipeline(
        steps=[
            ("preprocess", preprocess),
            ("model", model),
        ]
    )


def evaluate_ablation_feature_set(name: str, train_X: pd.DataFrame, val_X: pd.DataFrame) -> dict:
    pipeline = build_rf_pipeline_for_ablation(train_X)
    pipeline.fit(train_X, y_train)

    val_probs = pipeline.predict_proba(val_X)[:, 1]
    val_auc = roc_auc_score(y_val, val_probs)
    val_rmse = mean_squared_error(y_val, val_probs, squared=False)
    approval_rate_050 = (val_probs >= 0.5).mean()

    return {
        "Feature_Set": name,
        "Val_AUC": val_auc,
        "Val_RMSE": val_rmse,
        "Approval_Rate_050": approval_rate_050,
    }


feature_ablation_results = [
    evaluate_ablation_feature_set("FeatureSet_A (Full)", X_train_full, X_val_full),
    evaluate_ablation_feature_set("FeatureSet_B (No-Grade)", X_train_nograde, X_val_nograde),
]

feature_ablation_df = pd.DataFrame(feature_ablation_results)
feature_ablation_df["Val_AUC"] = feature_ablation_df["Val_AUC"].round(6)
feature_ablation_df["Val_RMSE"] = feature_ablation_df["Val_RMSE"].round(6)
feature_ablation_df["Approval_Rate_050"] = feature_ablation_df["Approval_Rate_050"].round(4)

feature_ablation_df


Sanity check: df_test not used in feature ablation (train/val only).
Dropping columns for FeatureSet_B: ['grade', 'sub_grade']


C:\Users\mail\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


C:\Users\mail\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,Feature_Set,Val_AUC,Val_RMSE,Approval_Rate_050
0,FeatureSet_A (Full),0.695395,0.462537,0.3968
1,FeatureSet_B (No-Grade),0.695576,0.458125,0.3703


**Decision:** FeatureSet_B (No-Grade) is selected. It slightly improves validation AUC (0.695576 vs 0.695395) and improves RMSE (0.458125 vs 0.462537) while removing internal grade labels, which improves interpretability without a performance penalty.


We select the final model for business thresholding based on validation AUC.


In [43]:
# Goal: Select the best-performing model (highest Validation AUC) among rubric models.
# Inputs: comparison_df and all trained pipelines.
# Outputs: FINAL_MODEL_NAME and FINAL_PIPELINE.
# Why it matters: Restricting selection to rubric models keeps grading consistent while preserving model comparison transparency.

rubric_comparison_df = comparison_df[comparison_df["role"] == "Rubric Model"].copy()
rubric_comparison_df = rubric_comparison_df.sort_values("Val_AUC", ascending=False).reset_index(drop=True)

FINAL_MODEL_NAME = rubric_comparison_df.loc[0, "Model"]

FINAL_PIPELINE = {
    "Logistic Regression": log_reg_pipeline,
    "Decision Tree": best_tree_pipeline,
    "Random Forest": rf_best_pipeline,
    "Gradient Boosting": gb_best_pipeline,
}[FINAL_MODEL_NAME]

FINAL_MODEL_NAME


'Random Forest'

## Threshold Optimization (Business)

Positive class is `default = 1`, and the model outputs P(default). We make a lending decision by comparing the predicted default probability to a threshold: **approve** if P(default) < threshold, **reject** otherwise. The optimal threshold depends on business costs and benefits. Thresholds are chosen on the validation set; the test set is reserved for final evaluation only.


### Business Assumptions

Because no monetary cost-benefit values are provided, we use a clear, outcome-based payoff matrix (defined in `PAYOFF`):

- Approve & non‑default: +1
- Approve & default: −5
- Reject & default: 0
- Reject & non‑default: −1

Defaults are much more costly than missed opportunities, so the policy is intentionally conservative. These values are illustrative but economically reasonable for decision‑making.


In [44]:
# Goal: Determine the optimal classification threshold that maximizes business value based on payoff assumptions.
# Inputs: FINAL_PIPELINE, validation set, and PAYOFF matrix.
# Outputs: optimal_table, best_threshold, and threshold sensitivity analysis.
# Why it matters: This translates a probability (ML) into a decision (Business), ensuring the model creates the most economic value.

val_proba = FINAL_PIPELINE.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.01, 0.99, 99)
results_threshold = []

for threshold in thresholds:
    approve = val_proba < threshold

    approve_nondefault = ((approve) & (y_val == 0)).sum()
    approve_default = ((approve) & (y_val == 1)).sum()
    reject_nondefault = ((~approve) & (y_val == 0)).sum()
    reject_default = ((~approve) & (y_val == 1)).sum()

    expected_value = (
        approve_nondefault * PAYOFF["approve_nondefault"]
        + approve_default * PAYOFF["approve_default"]
        + reject_default * PAYOFF["reject_default"]
        + reject_nondefault * PAYOFF["reject_nondefault"]
    )

    approval_rate = approve.mean()

    results_threshold.append(
        {
            "threshold": threshold,
            "expected_value": expected_value,
            "approval_rate": approval_rate,
            "approve_nondefault": approve_nondefault,
            "approve_default": approve_default,
            "reject_nondefault": reject_nondefault,
            "reject_default": reject_default,
        }
    )

threshold_df = pd.DataFrame(results_threshold)

best_idx = threshold_df["expected_value"].idxmax()
best_threshold = threshold_df.loc[best_idx, "threshold"]
best_expected_value = threshold_df.loc[best_idx, "expected_value"]
best_approval_rate = threshold_df.loc[best_idx, "approval_rate"]

# Expected value curve
ev_fig, ev_ax = plt.subplots(figsize=(7, 4))
ev_ax.plot(threshold_df["threshold"], threshold_df["expected_value"], marker="o", markersize=3)
ev_ax.axvline(best_threshold, color="red", linestyle="--", label=f"Best threshold = {best_threshold:.2f}")
ev_ax.set_title("Expected Value vs Threshold (Validation)")
ev_ax.set_xlabel("Threshold")
ev_ax.set_ylabel("Expected Value")
ev_ax.legend()
plt.show()

# Approval rate curve
approval_fig, approval_ax = plt.subplots(figsize=(7, 4))
approval_ax.plot(threshold_df["threshold"], threshold_df["approval_rate"], marker="o", markersize=3)
approval_ax.axvline(best_threshold, color="red", linestyle="--", label=f"Best threshold = {best_threshold:.2f}")
approval_ax.set_title("Approval Rate vs Threshold (Validation)")
approval_ax.set_xlabel("Threshold")
approval_ax.set_ylabel("Approval Rate")
approval_ax.legend()
plt.show()

# Confusion-style tables at 0.50 and optimal threshold

def decision_table(threshold):
    approve = val_proba < threshold
    approve_nondefault = int(((approve) & (y_val == 0)).sum())
    approve_default = int(((approve) & (y_val == 1)).sum())
    reject_nondefault = int(((~approve) & (y_val == 0)).sum())
    reject_default = int(((~approve) & (y_val == 1)).sum())

    table = pd.DataFrame(
        {
            "Non-default": [approve_nondefault, reject_nondefault],
            "Default": [approve_default, reject_default],
        },
        index=["Approved", "Rejected"],
    )

    approvals = approve.sum()
    bad_approvals = approve_default
    approval_rate = approvals / len(y_val)

    return table, approvals, bad_approvals, approval_rate

baseline_table, baseline_approvals, baseline_bad, baseline_rate = decision_table(0.50)
optimal_table, optimal_approvals, optimal_bad, optimal_rate = decision_table(best_threshold)

baseline_table, optimal_table
print(f"Baseline approvals: {baseline_approvals}, defaults among approved: {baseline_bad}, approval rate: {baseline_rate:.3f}")
print(f"Optimal approvals: {optimal_approvals}, defaults among approved: {optimal_bad}, approval rate: {optimal_rate:.3f}")

# Sensitivity analysis: scale approve outcomes by ±20%

sensitivity_rows = []
for multiplier in [0.8, 1.0, 1.2]:
    payoff = PAYOFF.copy()
    payoff["approve_nondefault"] = PAYOFF["approve_nondefault"] * multiplier
    payoff["approve_default"] = PAYOFF["approve_default"] * multiplier

    values = []
    approvals = []
    for threshold in thresholds:
        approve = val_proba < threshold
        approve_nondefault = ((approve) & (y_val == 0)).sum()
        approve_default = ((approve) & (y_val == 1)).sum()
        reject_nondefault = ((~approve) & (y_val == 0)).sum()
        reject_default = ((~approve) & (y_val == 1)).sum()

        ev = (
            approve_nondefault * payoff["approve_nondefault"]
            + approve_default * payoff["approve_default"]
            + reject_default * payoff["reject_default"]
            + reject_nondefault * payoff["reject_nondefault"]
        )

        values.append(ev)
        approvals.append(approve.mean())

    best_idx = int(np.argmax(values))
    sensitivity_rows.append(
        {
            "multiplier": multiplier,
            "best_threshold": thresholds[best_idx],
            "best_expected_value": values[best_idx],
            "best_approval_rate": approvals[best_idx],
        }
    )

sensitivity_df = pd.DataFrame(sensitivity_rows)

sensitivity_df


Baseline approvals: 1528, defaults among approved: 182, approval rate: 0.603
Optimal approvals: 1992, defaults among approved: 280, approval rate: 0.786


,multiplier,best_threshold,best_expected_value,best_approval_rate
0,0.8,0.58,-83.4,0.786419
1,1.0,0.58,-21.0,0.786419
2,1.2,0.58,41.4,0.786419


Sensitivity results show how the optimal threshold shifts when approval payoffs are scaled by ±20%. A stable threshold indicates robust decisioning; larger shifts indicate stronger dependence on business cost assumptions.


## Business Interpretation of Threshold Optimization

Under the baseline cost–benefit assumptions provided, the expected value (EV) remains negative across all thresholds. This result does not indicate a modeling error, but rather reflects conservative payoff assumptions in which the cost of a default outweighs the benefit of approving a performing loan.

In this setting, the optimal threshold is selected as the value that minimizes expected losses while balancing approval volume and default risk. Sensitivity analysis shows that small changes in the assumed cost of default or benefit of repayment can shift EV into positive territory, highlighting the dependence of lending decisions on business assumptions rather than purely statistical performance.

Accordingly, threshold optimization is interpreted as a **decision-support exercise**, illustrating how predicted default probabilities can be translated into actionable lending policies under different economic scenarios, rather than as a guarantee of positive profitability under a single fixed payoff structure.


In [45]:
# Goal: Visualize the Expected Value curve to identify the point of maximum profitability.
# Inputs: threshold_df results.
# Outputs: A visualization of Expected Value vs. Classification Threshold.
# Why it matters: This plot clearly shows the "sweet spot" where lending decisions are most profitable under current assumptions.

plt.figure(figsize=(7, 4))
plt.plot(threshold_df["threshold"], threshold_df["expected_value"], marker="o", markersize=3)
plt.axvline(best_threshold, color="red", linestyle="--", label=f"Best threshold = {best_threshold:.2f}")
plt.title("Expected Value vs Threshold (Validation)")
plt.xlabel("Classification Threshold")
plt.ylabel("Expected Value")
plt.legend()
plt.show()


**Interpretation**

The optimal threshold from the validation expected-value curve (reported above) reflects a more conservative lending policy, because default costs are much larger than missed repayment opportunities. If default costs were lower or opportunity costs higher, the optimal threshold would shift downward to approve more loans.


## Results & Takeaways

Below we summarize model performance, the final model choice, and the threshold policy used for business decisioning.


In [46]:
comparison_df

threshold_summary = pd.DataFrame(
    [
        {
            "final_model": FINAL_MODEL_NAME,
            "best_threshold": best_threshold,
            "best_expected_value": best_expected_value,
            "best_approval_rate": best_approval_rate,
        }
    ]
)
threshold_summary


,final_model,best_threshold,best_expected_value,best_approval_rate
0,Random Forest,0.58,-21,0.786419


- The final model is selected by highest validation AUC to maximize generalization performance.
- The AUC_Gap column provides an overfitting check; smaller gaps indicate more stable models.
- RMSE complements AUC by reflecting probability calibration quality.
- The optimal threshold balances approval volume and default risk under the payoff assumptions.
- Sensitivity analysis indicates how robust the threshold is to cost shifts.


In [47]:
# Goal: Print the versions of key libraries used in this project.
# Inputs: sys, sklearn, pandas, numpy modules.
# Outputs: Version printouts for reproducibility.
# Why it matters: Tracking versions ensures that others can recreate the exact same environment and results.

import sys
import sklearn

print("python_version:", sys.version.split()[0])
print("pandas_version:", pd.__version__)
print("numpy_version:", np.__version__)
print("sklearn_version:", sklearn.__version__)
print("random_state:", RANDOM_STATE)
print("train_shape:", X_train.shape, "val_shape:", X_val.shape)
print("target_mean_train:", y_train.mean(), "target_mean_val:", y_val.mean())



python_version: 3.12.6
pandas_version: 2.2.2
numpy_version: 2.1.1
sklearn_version: 1.5.2
random_state: 42
train_shape: (7596, 27) val_shape: (2533, 27)
target_mean_train: 0.19286466561348078 target_mean_val: 0.19265692854322938


## Final Test Evaluation

Final test evaluation is intentionally deferred to preserve an unbiased hold‑out set.


## Conclusion

This notebook follows the full supervised learning workflow for credit risk, with clear separation between exploration, modeling, and business decisioning.


## Policy Metrics Summary (Validation vs Test)

The table below summarizes approval and default outcomes for the locked thresholds on validation and test. It uses the existing confusion matrices from earlier steps and does not re-optimize thresholds.


In [ ]:
# Goal: Summarize policy metrics for validation vs test at locked thresholds.
# Inputs: Confusion matrices from validation/test (existing outputs or in-memory tables).
# Outputs: policy_metrics_df for display.
# Why it matters: Provides a side-by-side view of approval/default tradeoffs without re-optimizing thresholds.

threshold_opt = 0.58
if "best_threshold" in globals():
    assert np.isclose(best_threshold, threshold_opt), "Locked threshold must be 0.58"


def load_conf_table(var_name: str, path: Path) -> pd.DataFrame:
    if var_name in globals():
        return globals()[var_name].copy()
    return pd.read_csv(path, index_col=0)


def compute_policy_metrics(label: str, threshold: float, table: pd.DataFrame) -> dict:
    approved_nondefault = int(table.loc["Approved", "Non-default"])
    approved_default = int(table.loc["Approved", "Default"])
    rejected_nondefault = int(table.loc["Rejected", "Non-default"])
    rejected_default = int(table.loc["Rejected", "Default"])

    approvals = approved_nondefault + approved_default
    total = approved_nondefault + approved_default + rejected_nondefault + rejected_default

    approval_rate = approvals / total if total > 0 else np.nan
    default_rate_among_approved = approved_default / approvals if approvals > 0 else np.nan
    precision_default = default_rate_among_approved
    recall_default = rejected_default / (rejected_default + approved_default) if (rejected_default + approved_default) > 0 else np.nan

    return {
        "row": label,
        "threshold": threshold,
        "approval_rate": approval_rate,
        "default_rate_among_approved": default_rate_among_approved,
        "precision_default (defaults among approved)": precision_default,
        "recall_default (TPR on defaults)": recall_default,
        "TN": approved_nondefault,
        "FP": rejected_nondefault,
        "FN": approved_default,
        "TP": rejected_default,
    }


val_table_050 = load_conf_table("baseline_table", OUTPUT_DIR / "confusion_matrix_validation_050.csv")
val_table_opt = load_conf_table("optimal_table", OUTPUT_DIR / "confusion_matrix_validation_optimal.csv")

test_table_050 = load_conf_table("test_table_050", OUTPUT_DIR / "confusion_matrix_test_050.csv")
test_table_opt = load_conf_table("test_table_optimal", OUTPUT_DIR / "confusion_matrix_test_optimal.csv")

rows = [
    compute_policy_metrics("Validation @ 0.50", 0.50, val_table_050),
    compute_policy_metrics("Validation @ 0.58", threshold_opt, val_table_opt),
    compute_policy_metrics("Test @ 0.50", 0.50, test_table_050),
    compute_policy_metrics("Test @ 0.58", threshold_opt, test_table_opt),
]

policy_metrics_df = pd.DataFrame(rows).set_index("row")
policy_metrics_df[[
    "approval_rate",
    "default_rate_among_approved",
    "precision_default (defaults among approved)",
    "recall_default (TPR on defaults)",
]] = policy_metrics_df[[
    "approval_rate",
    "default_rate_among_approved",
    "precision_default (defaults among approved)",
    "recall_default (TPR on defaults)",
]].round(6)

policy_metrics_df


## Final Holdout Test Results (Single Use)

The test set was not used for model selection, hyperparameter tuning, feature selection, or threshold optimization. It is evaluated once at the end using the locked validation-chosen threshold.


In [48]:
# Goal: Evaluate the locked final model on the holdout test set exactly once.
# Inputs: df_test, X_train, X_val, y_train, y_val, and best_threshold from validation.
# Outputs: test_metrics, test_summary_df, test_table_050, test_table_optimal, and test curves.
# Why it matters: Provides an unbiased estimate of out-of-sample performance without leakage.

print("Sanity check: df_test is used only in this final holdout evaluation section.")

validation_best_threshold = float(best_threshold)
print(f"Validation-optimal threshold used on test: {validation_best_threshold:.4f}")
assert np.isclose(validation_best_threshold, best_threshold)

# Build full training set (train + validation) using FeatureSet_B (drop grade/sub_grade)
X_full_train = pd.concat([X_train, X_val], axis=0)
y_full_train = pd.concat([y_train, y_val], axis=0)

no_grade_cols = [c for c in ["grade", "sub_grade"] if c in X_full_train.columns]
X_full_train = X_full_train.drop(columns=no_grade_cols)

# Prepare test set (FeatureSet_B)
y_test = df_test[TARGET_COL].astype(int)
X_test = df_test.drop(columns=[TARGET_COL] + ([ID_COL] if ID_COL and ID_COL in df_test.columns else []))
X_test = add_engineered_features(X_test)
X_test = X_test.drop(columns=no_grade_cols, errors="ignore")

print(f"Test default prevalence: {y_test.mean():.4f}")

cat_cols_final = X_full_train.select_dtypes(include=["object"]).columns.tolist()
num_cols_final = [c for c in X_full_train.columns if c not in cat_cols_final]

numeric_transformer_final = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_transformer_final = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess_final = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_final, num_cols_final),
        ("cat", categorical_transformer_final, cat_cols_final),
    ]
)

final_rf_model = RandomForestClassifier(
    n_estimators=600,
    max_features="sqrt",
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=50,
    class_weight="balanced",
    bootstrap=True,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

final_test_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_final),
        ("model", final_rf_model),
    ]
)

final_test_pipeline.fit(X_full_train, y_full_train)

# Lock final pipeline for downstream exports (feature importance, artifacts)
FINAL_PIPELINE = final_test_pipeline
FINAL_MODEL_NAME = "Random Forest"

# Test predictions

test_proba = final_test_pipeline.predict_proba(X_test)[:, 1]

test_auc = roc_auc_score(y_test, test_proba)
test_rmse = mean_squared_error(y_test, test_proba, squared=False)


def test_decision_table(threshold: float):
    approve = test_proba < threshold

    approve_nondefault = int(((approve) & (y_test == 0)).sum())
    approve_default = int(((approve) & (y_test == 1)).sum())
    reject_nondefault = int(((~approve) & (y_test == 0)).sum())
    reject_default = int(((~approve) & (y_test == 1)).sum())

    table = pd.DataFrame(
        {
            "Non-default": [approve_nondefault, reject_nondefault],
            "Default": [approve_default, reject_default],
        },
        index=["Approved", "Rejected"],
    )

    approvals = int(approve.sum())
    approval_rate = approve.mean()
    default_rate_approved = (approve_default / approvals) if approvals > 0 else np.nan

    return table, approvals, approve_default, approval_rate, default_rate_approved


test_table_050, test_approvals_050, test_bad_050, test_rate_050, test_bad_rate_050 = test_decision_table(0.50)
(
    test_table_optimal,
    test_approvals_opt,
    test_bad_opt,
    test_rate_opt,
    test_bad_rate_opt,
) = test_decision_table(validation_best_threshold)

# Compact summary table

test_summary_df = pd.DataFrame(
    [
        {
            "Threshold_Label": "0.50",
            "Threshold": 0.50,
            "Approval_Rate": test_rate_050,
            "Default_Rate_Approved": test_bad_rate_050,
            "Approvals": test_approvals_050,
            "Defaults_Approved": test_bad_050,
            "Test_AUC": test_auc,
            "Test_RMSE": test_rmse,
        },
        {
            "Threshold_Label": "Optimal (Validation)",
            "Threshold": validation_best_threshold,
            "Approval_Rate": test_rate_opt,
            "Default_Rate_Approved": test_bad_rate_opt,
            "Approvals": test_approvals_opt,
            "Defaults_Approved": test_bad_opt,
            "Test_AUC": test_auc,
            "Test_RMSE": test_rmse,
        },
    ]
)

test_summary_df[["Approval_Rate", "Default_Rate_Approved", "Test_AUC", "Test_RMSE"]] = (
    test_summary_df[["Approval_Rate", "Default_Rate_Approved", "Test_AUC", "Test_RMSE"]].round(6)
)

test_summary_df

# Store metrics for export

test_metrics = {
    "test_auc": float(test_auc),
    "test_rmse": float(test_rmse),
    "threshold_050": {
        "threshold": 0.50,
        "approval_rate": float(test_rate_050),
        "default_rate_approved": float(test_bad_rate_050),
        "approvals": int(test_approvals_050),
        "defaults_approved": int(test_bad_050),
    },
    "threshold_optimal_validation": {
        "threshold": float(validation_best_threshold),
        "approval_rate": float(test_rate_opt),
        "default_rate_approved": float(test_bad_rate_opt),
        "approvals": int(test_approvals_opt),
        "defaults_approved": int(test_bad_opt),
    },
}

# Optional test curves (no threshold optimization on test)

test_threshold_results = []
for threshold in thresholds:
    approve = test_proba < threshold

    approve_nondefault = ((approve) & (y_test == 0)).sum()
    approve_default = ((approve) & (y_test == 1)).sum()
    reject_nondefault = ((~approve) & (y_test == 0)).sum()
    reject_default = ((~approve) & (y_test == 1)).sum()

    expected_value = (
        approve_nondefault * PAYOFF["approve_nondefault"]
        + approve_default * PAYOFF["approve_default"]
        + reject_default * PAYOFF["reject_default"]
        + reject_nondefault * PAYOFF["reject_nondefault"]
    )

    approval_rate = approve.mean()

    test_threshold_results.append(
        {
            "threshold": threshold,
            "expected_value": expected_value,
            "approval_rate": approval_rate,
        }
    )

test_threshold_df = pd.DataFrame(test_threshold_results)

# Expected value curve (Test)

test_ev_fig, test_ev_ax = plt.subplots(figsize=(7, 4))
test_ev_ax.plot(test_threshold_df["threshold"], test_threshold_df["expected_value"], marker="o", markersize=3)
test_ev_ax.axvline(validation_best_threshold, color="red", linestyle="--", label=f"Validation threshold = {validation_best_threshold:.2f}")
test_ev_ax.set_title("Expected Value vs Threshold (Test)")
test_ev_ax.set_xlabel("Threshold")
test_ev_ax.set_ylabel("Expected Value")
test_ev_ax.legend()
plt.show()

# Approval rate curve (Test)

test_approval_fig, test_approval_ax = plt.subplots(figsize=(7, 4))
test_approval_ax.plot(test_threshold_df["threshold"], test_threshold_df["approval_rate"], marker="o", markersize=3)
test_approval_ax.axvline(validation_best_threshold, color="red", linestyle="--", label=f"Validation threshold = {validation_best_threshold:.2f}")
test_approval_ax.set_title("Approval Rate vs Threshold (Test)")
test_approval_ax.set_xlabel("Threshold")
test_approval_ax.set_ylabel("Approval Rate")
test_approval_ax.legend()
plt.show()


Sanity check: df_test is used only in this final holdout evaluation section.
Validation-optimal threshold used on test: 0.5800
Test default prevalence: 0.1931


C:\Users\mail\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Interpretation (Test vs Validation):**
- Test AUC/RMSE are compared to validation to gauge generalization stability.
- The optimal threshold trades higher approval rates for higher defaults among approved loans.
- Results are directionally consistent with validation if the optimal threshold improves expected value while controlling defaults.
- The test evaluation is a single, final audit and does not change any modeling or threshold decisions.


## Export Artifacts (Optional)


In [49]:
# Goal: Export all key results, charts, and metadata for final delivery.
# Inputs: comparison_df, threshold_df, plots, and final pipeline.
# Outputs: Multiple CSV, PNG, and MD files in the "outputs" directory, plus a manifest.
# Why it matters: This final step packages the project artifacts, making them ready for use in reporting, auditing, or deployment.

import subprocess
import hashlib
import json

FINAL_OUTPUT_FILES = [
    "model_card.md",
    "final_report.md",
    "model_comparison_validation.csv",
    "tuning_decision_tree.csv",
    "tuning_random_forest.csv",
    "tuning_gradient_boosting.csv",
    "feature_importance_final_model.csv",
    "feature_importance_final_model.png",
    "ev_curve_validation.png",
    "approval_rate_curve_validation.png",
    "confusion_matrix_validation_050.csv",
    "confusion_matrix_validation_optimal.csv",
    "threshold_sweep_validation.csv",
    "threshold_sensitivity.csv",
    "test_metrics.json",
    "confusion_matrix_test_050.csv",
    "confusion_matrix_test_optimal.csv",
    "ev_curve_test.png",
    "approval_rate_curve_test.png",
]
MANIFEST_NAME = "outputs_manifest.txt"

OPTIONAL_APPENDIX_FILES = []

# Cleanup: remove existing known outputs only (clean export)
for name in FINAL_OUTPUT_FILES + [MANIFEST_NAME]:
    p = OUTPUT_DIR / name
    if p.exists():
        p.unlink()

# Export tables
comparison_df.to_csv(OUTPUT_DIR / "model_comparison_validation.csv", index=False)

dt_tuning_df.to_csv(OUTPUT_DIR / "tuning_decision_tree.csv", index=False)
rf_tuning_df.to_csv(OUTPUT_DIR / "tuning_random_forest.csv", index=False)
gb_tuning_df.to_csv(OUTPUT_DIR / "tuning_gradient_boosting.csv", index=False)

threshold_df.to_csv(OUTPUT_DIR / "threshold_sweep_validation.csv", index=False)
sensitivity_df.to_csv(OUTPUT_DIR / "threshold_sensitivity.csv", index=False)

baseline_table.to_csv(OUTPUT_DIR / "confusion_matrix_validation_050.csv")
optimal_table.to_csv(OUTPUT_DIR / "confusion_matrix_validation_optimal.csv")

# Test outputs (single-use holdout)
if "test_table_050" in globals():
    test_table_050.to_csv(OUTPUT_DIR / "confusion_matrix_test_050.csv")
if "test_table_optimal" in globals():
    test_table_optimal.to_csv(OUTPUT_DIR / "confusion_matrix_test_optimal.csv")
if "test_metrics" in globals():
    (OUTPUT_DIR / "test_metrics.json").write_text(
        json.dumps(test_metrics, indent=2),
        encoding="utf-8",
    )

# Export curves
if "ev_fig" in globals():
    ev_fig.savefig(OUTPUT_DIR / "ev_curve_validation.png", dpi=300, bbox_inches="tight")
if "approval_fig" in globals():
    approval_fig.savefig(OUTPUT_DIR / "approval_rate_curve_validation.png", dpi=300, bbox_inches="tight")
if "test_ev_fig" in globals():
    test_ev_fig.savefig(OUTPUT_DIR / "ev_curve_test.png", dpi=300, bbox_inches="tight")
if "test_approval_fig" in globals():
    test_approval_fig.savefig(OUTPUT_DIR / "approval_rate_curve_test.png", dpi=300, bbox_inches="tight")

# Feature importance for final model (no retraining)
feature_names = FINAL_PIPELINE.named_steps["preprocess"].get_feature_names_out()
model = FINAL_PIPELINE.named_steps["model"]
if hasattr(model, "feature_importances_"):
    importances = model.feature_importances_
elif hasattr(model, "coef_"):
    importances = np.abs(model.coef_).ravel()
else:
    importances = None

if importances is None:
    raise RuntimeError("Final model does not expose feature importances.")

fi_df = pd.DataFrame({"feature": feature_names, "importance": importances})
fi_df = fi_df.sort_values("importance", ascending=False)
fi_df.to_csv(OUTPUT_DIR / "feature_importance_final_model.csv", index=False)

plt.figure(figsize=(8, 4))
sns.barplot(
    x="importance",
    y="feature",
    data=fi_df.head(10),
    orient="h",
)
plt.title(f"Top 10 Feature Importances ({FINAL_MODEL_NAME})")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.savefig(OUTPUT_DIR / "feature_importance_final_model.png", dpi=300, bbox_inches="tight")
plt.show()

# Model card
final_row = comparison_df.loc[comparison_df["Model"] == FINAL_MODEL_NAME].iloc[0]
model_card = f"""# Model Card

## Executive Summary
- Predicts probability of default to support lending decisions.
- Final model selected by highest validation AUC for generalization.
- Threshold policy balances approval volume and default risk.
- Decisioning is grounded in transparent payoff assumptions.

## Data & Split
- Train rows: {X_train.shape[0]}
- Validation rows: {X_val.shape[0]}
- Test rows: {df_test.shape[0]}
- Target prevalence (train): {y_train.mean():.4f}
- Target prevalence (validation): {y_val.mean():.4f}
- Random state: {RANDOM_STATE}

## Feature Engineering
- Affordability: installment_to_income, loan_to_income
- Leverage: revol_balance_util, revol_balance_to_income
- Stability/experience: open_to_total_acc, recent_inquiry_flag
- Nonlinear: log_annual_inc, sqrt_dti

## Models Compared
Model performance is summarized in `model_comparison_validation.csv`. The final model is selected based on validation AUC, with AUC gap used to monitor overfitting risk.

## Final Policy (Validation)
- Final model: {FINAL_MODEL_NAME}
- Validation AUC: {final_row['Val_AUC']:.4f}
- Validation RMSE: {final_row['Val_RMSE']:.4f}
- Optimal threshold: {best_threshold:.2f}
- Approval rate: {best_approval_rate:.3f}
- Expected value: {best_expected_value}

## Confusion Matrices (Validation)
- Threshold 0.50: approvals={baseline_approvals}, defaults among approved={baseline_bad}, approval_rate={baseline_rate:.3f}
- Optimal threshold: approvals={optimal_approvals}, defaults among approved={optimal_bad}, approval_rate={optimal_rate:.3f}
- Full tables are saved in `confusion_matrix_validation_050.csv` and `confusion_matrix_validation_optimal.csv`.

## Assumptions & Interpretation
The decision policy is based on stylized payoff assumptions (e.g., -5 for a default and +1 for a successful repayment). Under these assumptions, the optimal threshold is chosen to minimize expected loss (or maximize expected value). It is important to note that the resulting Expected Value (EV) may be negative if the baseline default rate is high or if the model’s discriminative power is limited. The sensitivity analysis (`threshold_sensitivity.csv`) further explores how the optimal threshold and corresponding EV shift under different cost-benefit calibrations, highlighting the dependence of the policy on these business assumptions.

## Limitations & Next Steps
- Payoff assumptions are stylized; calibrate to bank economics.
- Threshold chosen on validation; test remains untouched for final audit.
- Monitor drift and recalibrate thresholds as conditions change.
- Consider calibration and fairness analyses as follow-on work.

## Reproducibility
- Run `notebooks/credit_risk_project.ipynb` → Restart Kernel & Run All.
"""
(OUTPUT_DIR / "model_card.md").write_text(model_card, encoding="utf-8")

# Final Report
final_report = f"""# Credit Risk Project Final Report

## Objective
This project builds a supervised machine learning model to estimate the probability of borrower default using historical loan data, enabling data-driven lending decisions.

## Data and Split Summary
- **Training Set**: {X_train.shape[0]} samples.
- **Validation Set**: {X_val.shape[0]} samples.
- **Stratification**: The split maintains the original target distribution (approx. {y_train.mean():.1%} default rate) in both sets.

## Leakage Controls
- **Target Separation**: The target variable (`{TARGET_COL}`) was removed from the features (`X`) before any model training.
- **Identifier Removal**: Unique identifiers (like `id`) were dropped to prevent models from "memorizing" specific rows.
- **Hold-out Test Set**: `lending_club_test.csv` is an unlabeled hold-out file, never used for training, validation, or threshold tuning. All model selection is performed using a stratified train/validation split from the training data only.

## Final Model and Performance
- **Final Model**: {FINAL_MODEL_NAME}
- **Validation AUC**: {final_row['Val_AUC']:.4f} (Measures ability to distinguish defaults from non-defaults)
- **AUC Gap**: {final_row['AUC_Gap']:.4f} (Difference between train and validation performance; monitors overfitting)
- **Validation RMSE**: {final_row['Val_RMSE']:.4f} (Measures probability calibration accuracy)

Detailed model comparisons can be found in [model_comparison_validation.csv](model_comparison_validation.csv).

## Threshold Decision Summary
To translate probabilities into approval/rejection decisions, we optimized the threshold to maximize expected value under conservative payoff assumptions (Defaults are 5x more costly than missed opportunities).

- **Optimal Threshold**: {best_threshold:.2f}
- **Expected Value**: {best_expected_value} (Total validation expected value under stylized payoff units: +1 approved non-default, −1 rejected non-default, −5 approved default, 0 rejected default)
- **Approval Rate**: {best_approval_rate:.1%} (Percentage of loans approved at this threshold)

## Key Visualizations

### Feature Importance
Shows the top 10 variables driving the model's predictions.
![Feature Importance](feature_importance_final_model.png)

### Expected Value Curve
Shows how business value changes across different probability thresholds.
![Expected Value Curve](ev_curve_validation.png)

### Approval Rate Curve
Shows the trade-off between the classification threshold and the percentage of loans approved.
![Approval Rate Curve](approval_rate_curve_validation.png)

## Exported Artifacts
- [Model Comparison Metrics](model_comparison_validation.csv)
- [Threshold Sweep Results](threshold_sweep_validation.csv)
- [Threshold Sensitivity Analysis](threshold_sensitivity.csv)
- [Confusion Matrix (Threshold=0.50)](confusion_matrix_validation_050.csv)
- [Confusion Matrix (Optimal Threshold)](confusion_matrix_validation_optimal.csv)
- [Model Card](model_card.md)
- [Outputs Manifest](outputs_manifest.txt)
"""
(OUTPUT_DIR / "final_report.md").write_text(final_report, encoding="utf-8")

# Outputs manifest (Deterministic: no timestamps, sorted files, git hash or unknown)
try:
    commit_hash = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:
    commit_hash = "unknown"

purposes = {
    "model_card.md": "Executive summary of model, metrics, and policy.",
    "final_report.md": "Comprehensive project summary report.",
    "model_comparison_validation.csv": "Validation metrics for all models.",
    "tuning_decision_tree.csv": "Decision Tree tuning grid results.",
    "tuning_random_forest.csv": "Random Forest tuning grid results.",
    "tuning_gradient_boosting.csv": "Gradient Boosting tuning grid results.",
    "feature_importance_final_model.csv": "Feature importances for final model.",
    "feature_importance_final_model.png": "Top-10 feature importance chart.",
    "ev_curve_validation.png": "Expected value vs threshold curve.",
    "approval_rate_curve_validation.png": "Approval rate vs threshold curve.",
    "confusion_matrix_validation_050.csv": "Confusion table at threshold=0.50.",
    "confusion_matrix_validation_optimal.csv": "Confusion table at optimal threshold.",
    "threshold_sweep_validation.csv": "Threshold sweep metrics on validation.",
    "threshold_sensitivity.csv": "Sensitivity of optimal threshold to payoff shifts.",
    "test_metrics.json": "Holdout test metrics (single-use evaluation).",
    "confusion_matrix_test_050.csv": "Test confusion table at threshold=0.50.",
    "confusion_matrix_test_optimal.csv": "Test confusion table at validation-optimal threshold.",
    "ev_curve_test.png": "Expected value vs threshold curve (test).",
    "approval_rate_curve_test.png": "Approval rate vs threshold curve (test).",
}

manifest_lines = [
    f"git_commit: {commit_hash}",
    "files:",
]

for name in sorted(FINAL_OUTPUT_FILES):
    path = OUTPUT_DIR / name
    if not path.exists():
        continue
    size_kb = path.stat().st_size / 1024
    with open(path, "rb") as f:
        sha256 = hashlib.sha256(f.read()).hexdigest()
    manifest_lines.append(f"- {name} | {size_kb:.1f} KB | {sha256} | {purposes.get(name, '')}")

(OUTPUT_DIR / MANIFEST_NAME).write_text("\n".join(manifest_lines), encoding="utf-8")
# Verification printout
print(f"OUTPUT_DIR: {OUTPUT_DIR.resolve()}")
exported_files = sorted([p for p in OUTPUT_DIR.iterdir() if p.is_file()])
for path in exported_files:
    size_kb = path.stat().st_size / 1024
    print(f"{path.name}: {size_kb:.1f} KB")
print("EXPORT COMPLETE")



OUTPUT_DIR: C:\Users\mail\OneDrive\Desktop\credit-risk-ml-mif\outputs
approval_rate_curve_test.png: 93.0 KB
approval_rate_curve_validation.png: 92.8 KB
confusion_matrix_test_050.csv: 0.1 KB
confusion_matrix_test_optimal.csv: 0.1 KB
confusion_matrix_validation_050.csv: 0.1 KB
confusion_matrix_validation_optimal.csv: 0.1 KB
ev_curve_test.png: 106.9 KB
ev_curve_validation.png: 106.5 KB
feature_importance_final_model.csv: 1.8 KB
feature_importance_final_model.png: 107.3 KB
final_report.md: 2.8 KB
model_card.md: 2.4 KB
model_comparison_validation.csv: 0.4 KB
outputs_manifest.txt: 2.8 KB
test_metrics.json: 0.5 KB
threshold_sensitivity.csv: 0.2 KB
threshold_sweep_validation.csv: 4.4 KB
tuning_decision_tree.csv: 0.7 KB
tuning_gradient_boosting.csv: 3.0 KB
tuning_random_forest.csv: 3.2 KB
EXPORT COMPLETE


In [50]:
# Goal: Display a neat table of filenames, sizes, and SHA256 hashes for reproducibility.
# Inputs: outputs/outputs_manifest.txt.
# Outputs: A formatted DataFrame showing file details.
# Why it matters: This provides immediate, visible proof that the files on disk match the manifest, ensuring the results are genuine and unchanged.

import pandas as pd
from pathlib import Path

manifest_path = OUTPUT_DIR / "outputs_manifest.txt"
if manifest_path.exists():
    with open(manifest_path, "r") as f:
        lines = f.readlines()
    
    file_data = []
    for line in lines:
        if line.startswith("- "):
            parts = line.strip("- ").strip().split(" | ")
            if len(parts) >= 3:
                file_data.append({
                    "Filename": parts[0].strip(),
                    "Size": parts[1].strip(),
                    "SHA256 Hash": parts[2].strip()
                })
    
    hash_df = pd.DataFrame(file_data)
    display(hash_df)
else:
    print("Manifest not found. Run the Export cell first.")


,Filename,Size,SHA256 Hash
0,approval_rate_curve_test.png,93.0 KB,654977128c4d86ccc561f0d1ba8057daba67887e054eb1...
1,approval_rate_curve_validation.png,92.8 KB,ff91a28cb10c087ac9b9cf860a1e11eadf26aacb8021b0...
2,confusion_matrix_test_050.csv,0.1 KB,ee16467e4af7ac32cd7f9a52b4deab101aebe820de4542...
3,confusion_matrix_test_optimal.csv,0.1 KB,dd0302e4d31019429c685e154ef260c6d07a8f23f3161d...
4,confusion_matrix_validation_050.csv,0.1 KB,7ec55cded958a019f6142dbc158d7eca734cad9837d54b...
5,confusion_matrix_validation_optimal.csv,0.1 KB,d6d8f50fe25902bef88267271b9bf74a63e7afc2b51017...
6,ev_curve_test.png,106.9 KB,2527d52e80397c232f14eacd6f986134bdcbd1f54d7479...
7,ev_curve_validation.png,106.5 KB,d6642929b3ed3fd58a61f9ff0d9d2c6d23f6a2f7663534...
8,feature_importance_final_model.csv,1.8 KB,7e57ea57df6a3db7902d2cc5424342fd4ebb2a2a60d8cf...
9,feature_importance_final_model.png,107.3 KB,745bbefe1d4edb5c44ff33f6c10a34f89613a02d9f2ff5...
